In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import math
import operator
from random import randint

csv_delim = ','
num_samples = 2000
test_sample = int(0.30 * num_samples)
num_features = 20
positive = 1
negative = 0

In [2]:
# set extraComma = True if csv is of the format: data-1, data-2, ..., data-n, (with additional comma at the end)
def open_with_panda(filename, extraComma):
    data = pd.read_csv(filename, sep=csv_delim, header=None, skip_blank_lines=True)
    if extraComma:
        return data.as_matrix()[:,:-1]
    else:
        return data.as_matrix()


# Filename for the DS1 covariance, mean 0, and mean 1
covariance_file = "hwk2_datasets_corrected/DS1_Cov.txt"
mean_0_file = "hwk2_datasets_corrected/DS1_m_0.txt"
mean_1_file = "hwk2_datasets_corrected/DS1_m_1.txt"

DS1_training = "DS1/training.csv"
DS1_test = "DS1/test.csv"

In [3]:
mean_data_0 = open_with_panda(mean_0_file, True).flatten()
mean_data_1 = open_with_panda(mean_1_file, True).flatten()
covariance_matrix = open_with_panda(covariance_file, True)

In [4]:
def generateData(mean, covariance, target, samples):
    data = np.random.multivariate_normal(mean, covariance, samples)
    target_matrix = np.full((samples, 1), target)
    return np.append(data, target_matrix, axis=1)


if not (os.path.isfile(DS1_training) and os.path.isfile(DS1_test)):
    normal_data_0 = generateData(mean_data_0, covariance_matrix, negative, num_samples)
    normal_data_1 = generateData(mean_data_1, covariance_matrix, positive, num_samples)

    testing_0 = normal_data_0[:test_sample,:]
    testing_1 = normal_data_1[:test_sample,:]

    test_data = np.append(testing_0, testing_1, axis=0)

    training_0 = normal_data_0[test_sample:, :]
    training_1 = normal_data_1[test_sample:, :]

    training_data = np.append(training_0, training_1, axis=0)

    np.savetxt(DS1_training, training_data, delimiter=csv_delim)
    np.savetxt(DS1_test, test_data, delimiter=csv_delim)



In [5]:
def sigmoid(a):
    i = 1 + (math.e ** (-a))
    return i ** -1


def class_probability(training_targets, target):
    N = len(training_targets)
    N_Class = sum(training_targets) # training_targets is an array of 0s and 1s, so we can just sum the ones

    if target == positive:
        return N_Class / N # We are after all the 1s / total points
    else:
        return (N - N_Class) / N # We are after 0s / total points; Number of 0s is N - N_Class (number of 1s)


def sample_means(input, output, desired_mean):
    if desired_mean == positive:
        N_1 = sum(output)
        x = np.zeros(len(input[0])) # [0,0,0,0...] with length the number of features

        for n in range(len(output)):
            t = output[n]
            x_n = input[n]
            x = x + t*x_n
        return x / N_1
    else: # We are looking for u_0, the negative class mean
        N_0 = len(output) - sum(output)
        x = np.zeros(len(input[0])) # [0,0,0,0...] with length the number of features

        for n in range(len(output)):
            t = output[n]
            x_n = input[n]
            x = x + (1-t)*x_n
        return x / N_0


def covariance(input, output, u_0, u_1):
    features = len(input[0])

    N = len(output)
    N_1 = sum(output)
    N_0 = N - N_1

    s1 = np.zeros((features,features))
    s0 = np.zeros((features,features))

    for i in range(len(output)):
        if output[i] == positive:
            s1 += np.dot((input[i] - u_1).reshape(-1,1), (input[i] - u_1).reshape(1,-1))
        elif output[i] == negative:
            s0 += np.dot((input[i] - u_0).reshape(-1,1), (input[i] - u_0).reshape(1,-1))

    s1 = s1 / N_1
    s0 = s0 / N_0

    return (N_1 / N) * s1 + (N_0 / N) * s0


def confusion(w_0, w, test_input, test_output):
    conf = np.zeros((2,2))

    for i in range(len(test_output)):
        prob_1 = sigmoid(np.dot(w.reshape(1,-1), test_input[i]) + w_0)

        # Predict success
        if prob_1 > (1 - prob_1):
            if test_output[i] == positive:
                conf[0][0] += 1
            elif test_output[i] == negative:
                conf[0][1] += 1
        # Predict Negative
        else:
            if test_output[i] == positive:
                conf[1][0] += 1
            elif test_output[i] == negative:
                conf[1][1] += 1
    return conf



In [6]:
data_for_training = open_with_panda(DS1_training, False)
input = data_for_training[:,:-1]
output = data_for_training[:,-1].flatten()

features = len(input[0])

p_1 = class_probability(output, positive)
p_0 = class_probability(output, negative)

u_1 = sample_means(input, output, positive)
u_0 = sample_means(input, output, negative)

cov = covariance(input, output, u_0, u_1)

w_0 = (-1/2) * np.dot(u_1.T, np.dot(np.linalg.inv(cov),u_1)) + (1/2) * np.dot(u_0.T, np.dot(np.linalg.inv(cov),u_0)) + math.log(p_1 / p_0)
w = np.dot(np.linalg.inv(cov), u_1 - u_0)

print(w_0)
print("[")
for i in range(len(w)):
    print(str(i) + ": " + str([w[i]]))
print("]")


-28.5636438698
[
0: [-14.97958970904708]
1: [9.0896749013246954]
2: [6.3206668629207439]
3: [3.4597761029438026]
4: [10.146994330221869]
5: [4.2188263350499344]
6: [-18.265797747673204]
7: [25.063182529460299]
8: [30.419582109649703]
9: [-9.346558085086432]
10: [13.840284382503954]
11: [13.516834721154789]
12: [-16.684278842601241]
13: [-13.75113699664751]
14: [6.1327979825897998]
15: [-13.66858669031415]
16: [-31.075980710459266]
17: [6.8884320363175853]
18: [0.88534748424387821]
19: [5.1596672086296262]
]


In [7]:
data_for_testing = open_with_panda(DS1_test, False)
input_test = data_for_testing[:,:-1]
output_test = data_for_testing[:,-1].flatten()

confusion_matrix = confusion(w_0, w, input_test, output_test)

precision = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[0][1])
recall = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[1][0])
accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1]) / (confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[1][0] + confusion_matrix[1][1])
f1 = (2 * precision * recall) / (precision + recall)

print(accuracy)
print(recall)
print(precision)
print(f1)

0.955833333333
0.96
0.952066115702
0.95601659751


In [8]:
# k-NN

def distance(x_1, x_2):
    x_3 = x_2 - x_1
    dis_squared = np.dot(x_3.reshape(1,-1), x_3.reshape(-1,1))
    return math.sqrt(dis_squared)


def confusionkNN(pred_output, test_output):
    conf = np.zeros((2,2))

    for i in range(len(test_output)):

        # Predict success
        if pred_output[i] == positive:
            if test_output[i] == positive:
                conf[0][0] += 1
            elif test_output[i] == negative:
                conf[0][1] += 1
        # Predict Negative
        else:
            if test_output[i] == positive:
                conf[1][0] += 1
            elif test_output[i] == negative:
                conf[1][1] += 1
    return conf


def getPrediction(training_inputs, training_outputs, test_input, kNN):
    distances = []
    for i in range(len(training_outputs)):
        d = distance(test_input, training_inputs[i])
        distances.append((d, training_outputs[i]))
    distances.sort(key=operator.itemgetter(0))
    nearest_neighbours = []
    for j in range(kNN):
        nearest_neighbours.append(distances[j][1])
    sum_elements = sum(nearest_neighbours)
    if kNN/2.0 < sum_elements:
        return positive
    elif kNN/2.0 > sum_elements:
        return negative
    else:
        return randint(0,1)



In [9]:
k_neighbours = 50
bestAccuracy = -1
bestk = -1

for k in range(1, k_neighbours, 2):
    print(k)
    pred_output = np.zeros((len(output_test),1))
    for i in range(len(output_test)):
        prediction = getPrediction(input, output, input_test[i], k)
        pred_output[i] = prediction

    confusion_matrix_kNN = confusionkNN(pred_output,output_test)
    accuracy = (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[1][1]) / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1] + confusion_matrix_kNN[1][0] + confusion_matrix_kNN[1][1])
    print(accuracy)
    if accuracy > bestAccuracy:
        bestK = k
        bestAccuracy = accuracy
        
print(bestK)  

1
0.525
3
0.519166666667
5
0.513333333333
7
0.525833333333
9
0.523333333333
11
0.5075
13
0.525
15
0.528333333333
17
0.529166666667
19
0.521666666667
21
0.529166666667
23
0.538333333333
25
0.5325
27
0.540833333333
29
0.538333333333
31
0.535833333333
33
0.541666666667
35
0.543333333333
37
0.530833333333
39
0.53
41
0.528333333333
43
0.529166666667
45
0.541666666667
47
0.544166666667
49
0.535833333333
47


In [10]:
pred_output = np.zeros((len(output_test),1))
for i in range(len(output_test)):
    prediction = getPrediction(input, output, input_test[i], 47)
    pred_output[i] = prediction

confusion_matrix_kNN = confusionkNN(pred_output,output_test)

In [11]:
precision = confusion_matrix_kNN[0][0] / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1])
recall = confusion_matrix_kNN[0][0] / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[1][0])
accuracy = (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[1][1]) / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1] + confusion_matrix_kNN[1][0] + confusion_matrix_kNN[1][1])
f1 = (2 * precision * recall) / (precision + recall)

print(accuracy)
print(recall)
print(precision)
print(f1)

0.544166666667
0.553333333333
0.543371522095
0.548307184145


#### kNN

In this situation, kNN did worse than LDA by far. It seemed like larger k values were marginally more accurate than smaller k values.

The performance indicators are above



# DS2

In [12]:
# DS2
# 2000 points per class - 4000 points total
# 400 points for 1: 200 for positive C1, 200 for negative C2
# 1680 points for 2: 840 for positive C1, 840 for negative C2
# 1920 points for 3: 960 for positive C1, 960 for negative C2

DS2_training = "DS2/training.csv"
DS2_test = "DS2/test.csv"

c1_mean_file = "hwk2_datasets_corrected/DS2_c1_m{input}.txt"
c2_mean_file = "hwk2_datasets_corrected/DS2_c2_m{input}.txt"
cov_file = "hwk2_datasets_corrected/DS2_Cov{input}.txt"

probabilities = {1:0.1, 2:0.42, 3:0.48}
proportions = {1:0, 2:0, 3:0}

# Convert the probabilities in to number of samples from each class
for i in range(num_samples):
    x = randint(1,100)
    if x <= probabilities[1]*100:
        proportions[1] += 1
    elif x <= (probabilities[1]*100 + probabilities[2]*100):
        proportions[2] += 1
    else:
        proportions[3] += 1
    
print(proportions)

{1: 201, 2: 840, 3: 959}


In [13]:
# The 0.3*proportions may not be exact - so we'll ensure test sample has 1200 points
test_proportions = {1:0, 2:0, 3:0}
test_proportions[1] = round(proportions[1]*0.3)
test_proportions[2] = round(proportions[2]*0.3)
test_proportions[3] = round(proportions[3]*0.3)

print(test_proportions)
print(test_proportions[1] + test_proportions[2] + test_proportions[3])

{1: 60, 2: 252, 3: 288}
600


In [17]:
if not (os.path.isfile(DS2_training) and os.path.isfile(DS2_test)):
    training_data = None
    test_data = None

    # Means 1 first
    i = 1
    proportion = proportions[i]
    mean_pos = open_with_panda(c1_mean_file.replace("{input}",str(i)), True).flatten()
    mean_neg = open_with_panda(c2_mean_file.replace("{input}", str(i)), True).flatten()
    cov_matrix = open_with_panda(cov_file.replace("{input}",str(i)), True)

    normal_data_0 = generateData(mean_neg, cov_matrix, negative, int(proportion))
    normal_data_1 = generateData(mean_pos, cov_matrix, positive, int(proportion))

    test_sample = test_proportions[i]

    testing_0 = normal_data_0[:test_sample,:]
    testing_1 = normal_data_1[:test_sample,:]

    test_data1 = np.append(testing_0, testing_1, axis=0)

    training_0 = normal_data_0[test_sample:, :]
    training_1 = normal_data_1[test_sample:, :]

    training_data1 = np.append(training_0, training_1, axis=0)

    # Means 2
    i = 2
    proportion = proportions[i]
    mean_pos = open_with_panda(c1_mean_file.replace("{input}",str(i)), True).flatten()
    mean_neg = open_with_panda(c2_mean_file.replace("{input}", str(i)), True).flatten()
    cov_matrix = open_with_panda(cov_file.replace("{input}",str(i)), True)

    normal_data_0 = generateData(mean_neg, cov_matrix, negative, int(proportion))
    normal_data_1 = generateData(mean_pos, cov_matrix, positive, int(proportion))

    test_sample = test_proportions[i]

    testing_0 = normal_data_0[:test_sample,:]
    testing_1 = normal_data_1[:test_sample,:]

    test_data2 = np.append(testing_0, testing_1, axis=0)

    training_0 = normal_data_0[test_sample:, :]
    training_1 = normal_data_1[test_sample:, :]

    training_data2 = np.append(training_0, training_1, axis=0)

    # Means 3
    i = 3
    proportion = proportions[i]
    mean_pos = open_with_panda(c1_mean_file.replace("{input}",str(i)), True).flatten()
    mean_neg = open_with_panda(c2_mean_file.replace("{input}", str(i)), True).flatten()
    cov_matrix = open_with_panda(cov_file.replace("{input}",str(i)), True)

    normal_data_0 = generateData(mean_neg, cov_matrix, negative, int(proportion))
    normal_data_1 = generateData(mean_pos, cov_matrix, positive, int(proportion))

    test_sample = test_proportions[i]

    testing_0 = normal_data_0[:test_sample,:]
    testing_1 = normal_data_1[:test_sample,:]

    test_data3 = np.append(testing_0, testing_1, axis=0)

    training_0 = normal_data_0[test_sample:, :]
    training_1 = normal_data_1[test_sample:, :]

    training_data3 = np.append(training_0, training_1, axis=0)

    training_data = np.append(training_data1, np.append(training_data2,training_data3, axis=0), axis=0)
    test_data = np.append(test_data1, np.append(test_data2,test_data3, axis=0), axis=0)

    np.savetxt(DS2_training, training_data, delimiter=csv_delim)
    np.savetxt(DS2_test, test_data, delimiter=csv_delim)


In [18]:
data2_for_training = open_with_panda(DS2_training, False)
input2 = data2_for_training[:,:-1]
output2 = data2_for_training[:,-1].flatten()

features = len(input2[0])

p2_1 = class_probability(output2, positive)
p2_0 = class_probability(output2, negative)

u2_1 = sample_means(input2, output2, positive)
u2_0 = sample_means(input2, output2, negative)

cov2 = covariance(input2, output2, u2_0, u2_1)

w2_0 = (-1/2) * np.dot(u2_1.T, np.dot(np.linalg.inv(cov2),u2_1)) + (1/2) * np.dot(u2_0.T, np.dot(np.linalg.inv(cov2),u2_0)) + math.log(p2_1 / p2_0)
w2 = np.dot(np.linalg.inv(cov2), u2_1 - u2_0)

print(w2_0)
print("[")
for i in range(len(w2)):
    print(str(i) + ": " + str([w2[i]]))
print("]")

data2_for_testing = open_with_panda(DS2_test, False)
input2_test = data2_for_testing[:,:-1]
output2_test = data2_for_testing[:,-1].flatten()

confusion_matrix = confusion(w2_0, w2, input2_test, output2_test)

precision = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[0][1])
recall = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[1][0])
accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1]) / (confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[1][0] + confusion_matrix[1][1])
f1 = (2 * precision * recall) / (precision + recall)

print("Accuracy " + str(accuracy))
print("Recall " + str(recall))
print("Precision " + str(precision))
print("F1 " + str(f1))

0.0576514093926
[
0: [-0.0022240563725057783]
1: [-0.0055789097410681998]
2: [-0.0027194834650929034]
3: [0.014551468683540278]
4: [-0.052670541086871719]
5: [0.063582391488058235]
6: [-0.036255056729106144]
7: [-0.012813378777203205]
8: [-0.010377617774236916]
9: [-0.049340719065959493]
10: [-0.012347150055690453]
11: [0.035082618976222613]
12: [-0.020998269767416103]
13: [0.068007298472954669]
14: [-0.010435103349685587]
15: [0.016446395582976156]
16: [-0.039968549068858855]
17: [0.021512193718622674]
18: [-0.01998561904617599]
19: [0.0031784602777031601]
]
Accuracy 0.515833333333
Recall 0.526666666667
Precision 0.515497553018
F1 0.521022258862


In [19]:
k_neighbours = 100
bestAccuracy = -1
bestkAccuracy = -1

bestPrec = -1
bestkPrecision = -1

for k in range(1, k_neighbours, 2):
    pred_output = np.zeros((len(output2_test), 1))
    for i in range(len(output2_test)):
        prediction = getPrediction(input2, output2, input2_test[i], k)
        pred_output[i] = prediction

    confusion_matrix_kNN = confusionkNN(pred_output, output2_test)
    accuracy = (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[1][1]) / (
            confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1] + confusion_matrix_kNN[1][0] +
            confusion_matrix_kNN[1][1])
    precision = confusion_matrix_kNN[0][0] / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1])
    if accuracy > bestAccuracy:
        bestkAccuracy = k
        bestAccuracy = accuracy
        print("Acc" + str(accuracy))
    if precision > bestPrec:
        bestPrec = precision
        bestkPrecision = k
        print("Prec" + str(precision))

print(bestkAccuracy)
print(bestkPrecision)

Acc0.486666666667
Prec0.486970684039
Acc0.505833333333
Prec0.506172839506
Acc0.513333333333
Prec0.514035087719
Acc0.5175
Prec0.51832460733
Acc0.519166666667
Prec0.520646319569
Acc0.52
Prec0.521582733813
83
83


In [20]:
neighbours = 83

pred_output = np.zeros((len(output_test),1))
for i in range(len(output_test)):
    prediction = getPrediction(input2, output2, input2_test[i], neighbours)
    pred_output[i] = prediction

confusion_matrix_kNN = confusionkNN(pred_output,output2_test)

precision = confusion_matrix_kNN[0][0] / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1])
recall = confusion_matrix_kNN[0][0] / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[1][0])
accuracy = (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[1][1]) / (confusion_matrix_kNN[0][0] + confusion_matrix_kNN[0][1] + confusion_matrix_kNN[1][0] + confusion_matrix_kNN[1][1])
f1 = (2 * precision * recall) / (precision + recall)

print(accuracy)
print(recall)
print(precision)
print(f1)

0.52
0.483333333333
0.521582733813
0.501730103806
